In [1]:
full_node_db = '/Users/karlkim/.chia/simulator/cats/db/blockchain_v2_simulator0.sqlite'
wallet_db = '/Users/karlkim/.chia/simulator/cats/wallet/db/blockchain_wallet_v2_r1_simulator0_236020623.sqlite'
chia_root = Path("/Users/karlkim/.chia/simulator/cats")
chia_config = load_config(chia_root, "config.yaml")
self_hostname = chia_config["self_hostname"]
full_node_rpc_port = chia_config["full_node"]["rpc_port"]
wallet_rpc_port = chia_config["wallet"]["rpc_port"]

In [2]:
@with_db_connection(full_node_db)
async def get_hints(conn):    
    query = "SELECT coin_id, hint from hints"
    async with conn.execute(query) as cursor:
        async for row in cursor:
            coin_id = row[0]
            hint = row[1]
            addr = encode_puzzle_hash(hint, 'txch')
            print(f'coin_id: {coin_id.hex()}')
            print(f'hint:\t {hint.hex()}')
            print(f'addr:\t {addr}\n')

@with_db_connection(wallet_db)
async def get_coins_by_wallet_id(conn, wallet_id):
    coins = []
    
    query = "SELECT coin_parent, puzzle_hash, amount FROM coin_record WHERE wallet_id=?"
    async with conn.execute(query, (wallet_id,)) as cursor:
        async for row in cursor:
            coin = Coin(bytes32.from_hexstr(row[0]), bytes32.from_hexstr(row[1]), uint64.from_bytes(row[2]))
            coins.append(coin)
    return coins

from chia.wallet.lineage_proof import LineageProof
@with_db_connection(wallet_db)
async def get_lineage_proof(conn, table_name):
    
    query = f"SELECT * FROM {table_name}"
    async with conn.execute(query) as cursor:
        async for row in cursor:
            ret = LineageProof.from_bytes(row[1])
            print(f'parent_name: {ret.parent_name.hex()}')
            print(f'inner_ph: {ret.inner_puzzle_hash.hex()}')
            print(f'amount: {ret.amount}')

@with_full_node_rpc_client(self_hostname, full_node_rpc_port, chia_root, chia_config)
async def get_coin_records_by_puzzle_hash(full_node_client, puzzle_hash):
    puzzle_hash_bytes: bytes32 = bytes32.from_hexstr(puzzle_hash)
    coin_records = await full_node_client.get_coin_records_by_puzzle_hash(
        puzzle_hash_bytes
    )
    return coin_records

@with_full_node_rpc_client(self_hostname, full_node_rpc_port, chia_root, chia_config)
async def get_coin_spend(full_node_client, coin_id, spent_height):
    coin_spend = await full_node_client.get_puzzle_and_solution(coin_id, spent_height)
    return coin_spend

@with_full_node_rpc_client(self_hostname, full_node_rpc_port, chia_root, chia_config)
async def get_coins_in_block(full_node_client, header_hash):
    result = await full_node_client.get_additions_and_removals(header_hash)
    return result

In [3]:
%%bash
chia version
cdv --version
python --version
chia show -s
chia wallet show -f 236020623
chia keys derive -f 236020623 wallet-address -n 5

1.7.0b3.dev114
cdv, version 1.1.4
Python 3.10.9
Network: simulator0    Port: 50090   RPC Port: 16909
Node ID: 9d1c999ccfb4313223a6af62695026a0a3d0ce5c3ca8034283b3ded7651f1007
Genesis Challenge: eb8c4d20b322be8d9fddbf9412016bdffe9a2901d7edb0e364e94266d0e095f7
Current Blockchain Status: Full Node Synced

Peak: Hash: 03fb75348b0ed95f0791c821a714edcbd8ad551402e330b296b8cfd5062e62c8
      Time: Wed Jan 11 2023 09:10:33 CST                  Height:          4

Estimated network space: 39.887 MiB
Current difficulty: 1024
Current VDF sub_slot_iters: 1024

  Height: |   Hash:
        4 | 03fb75348b0ed95f0791c821a714edcbd8ad551402e330b296b8cfd5062e62c8
        3 | 3f3dbf30047447cb9a11bd8707f3af5f0f93663a0157c10c9494d239a7efb24e
        2 | d690e9eee15e67078c6a09abebcdbe690622076261d7318bd2f91324d4c2ceb3
        1 | b4abff61105de1aaf1a3b527d776b7cff44d0710a46997e42623c6904f13e4cf
Wallet height: 4
Sync status: Synced
Balances, fingerprint: 236020623

Chia Wallet:
   -Total Balance:         5.0 txc

## Create Single-Issuance CAT
```sh
❯ cats --tail ./reference_tails/genesis_by_coin_id.clsp.hex --send-to $single_addr --amount 100000000000 -m 100000 --curry 0x0912e598c7e197a296283436bde9594fdc40fbaf9303cc842888d242f6af0cd5
...
Asset ID: a1cdd796a588491663f61dfe9305eb67e8fed0e016961da3958862c68589fd41
Eve Coin ID: 9faaec9124a3538cb7977ceea984bdb023914ac1e7bfa0329e3fa11ebbfe1e1a
```

In [7]:
genesis_coin_id = bytes32.from_hexstr("0x0912e598c7e197a296283436bde9594fdc40fbaf9303cc842888d242f6af0cd5")
genesis_by_coin_id = load_hex("./reference_tails/genesis_by_coin_id.clsp.hex")
curried_genesis_by_coin_id = genesis_by_coin_id.curry(genesis_coin_id)
asset_id = curried_genesis_by_coin_id.get_tree_hash()
asset_id.hex()

'a1cdd796a588491663f61dfe9305eb67e8fed0e016961da3958862c68589fd41'

## Examine CoinSpends
```sh

spent coins
  (0xd359601e8310a678e7ed914894cebe8f4c804fe1b4b0e3726d426e343e27c946 0x1cb70b7a0b0670102d0aea6db180ffe2375cea7e685fcebfd8f54482e0d5049f 0x048c27395000)
      => spent coin id 0912e598c7e197a296283436bde9594fdc40fbaf9303cc842888d242f6af0cd5

created coins
  (0x9faaec9124a3538cb7977ceea984bdb023914ac1e7bfa0329e3fa11ebbfe1e1a 0xbe9fc5a1356babff14efd8eb7ffaf152f221c402d71fcca0f0d216cd129c1c70 0x174876e800)
      => created coin id 01fc8146a15d1c089154902b8703852a7e0a42f151d2bbfa87e276e845d1a7de
  (0x0912e598c7e197a296283436bde9594fdc40fbaf9303cc842888d242f6af0cd5 0x52b7e40c813355fbc9393c08bb3c472584f0dd8c56910c50d20e538760899e12 0x0474dec0e160)
      => created coin id d6b74260b3be7a8ee7a0e09b9463817d4d80e3d3b2733adb9bba4f6f4c1f860d

ephemeral coins
  (0x0912e598c7e197a296283436bde9594fdc40fbaf9303cc842888d242f6af0cd5 0x71f0c6b06309d6582e247b523fd42d8cedf6e3c69f583619a2dbc9dda80827a2 0x174876e800)
      => created coin id 9faaec9124a3538cb7977ceea984bdb023914ac1e7bfa0329e3fa11ebbfe1e1a
created coin announcements
  ['0x9faaec9124a3538cb7977ceea984bdb023914ac1e7bfa0329e3fa11ebbfe1e1a', '0xcbf68bf42766be67fd42d52d263e401d314455b43d83f072555c6d5927508d9090'] =>
      51a5ad5671e227b6d1f3e0e1757cd4ff0874587d5a38f805d9733358bbaee763
  ['0x0912e598c7e197a296283436bde9594fdc40fbaf9303cc842888d242f6af0cd5', '0xcfe081ecf6778c82af2cdb60831508d7edcc4167da046771edbafbf7541f5b40'] =>
      b1895c47f027b323e7ed1597c08e6340dfa3c938ce2546226ffc2c21dc583c40


zero_coin_set = []

created  coin announcements = ['51a5ad5671e227b6d1f3e0e1757cd4ff0874587d5a38f805d9733358bbaee763', 'b1895c47f027b323e7ed1597c08e6340dfa3c938ce2546226ffc2c21dc583c40']

asserted coin announcements = ['51a5ad5671e227b6d1f3e0e1757cd4ff0874587d5a38f805d9733358bbaee763']

symdiff of coin announcements = ['b1895c47f027b323e7ed1597c08e6340dfa3c938ce2546226ffc2c21dc583c40']


================================================================================

aggregated signature check pass: True
pks: [<G1Element af14d22fba1ab9a63ec96f2f53cc4039fba865ab73753fb6ac9f8d397c864eb60349677ab155c355ed47f9a3e8e7bbaf>]
msgs: ['8699e090da8384d47c4120d9b79cf3812a1dbf655662d35081558a17d40e5c2f0912e598c7e197a296283436bde9594fdc40fbaf9303cc842888d242f6af0cd5ccd5bb71183532bff220ba46c268991a3ff07eb358e8255a65c30a2dce0e5fbb']
  msg_data: ['8699e090da8384d47c4120d9b79cf3812a1dbf655662d35081558a17d40e5c2f']
  coin_ids: ['0912e598c7e197a296283436bde9594fdc40fbaf9303cc842888d242f6af0cd5']
  add_data: ['ccd5bb71183532bff220ba46c268991a3ff07eb358e8255a65c30a2dce0e5fbb']
signature: b39708f1f89684a0034a6c2d800d3f8ea2ffe1acbd9b32a287cb5440f8ad8d416edbda99647ecd26c64db69f96e4b4e0149851c1a63b048b62eec2abebc36f9ecf2a388fc19826a974184560409a5c2e386d60d31776544ef3b4fd1e8876eeeb
None
```

In [13]:
genesis_coin_spend = await get_coin_spend(bytes32.from_hexstr("0x0912e598c7e197a296283436bde9594fdc40fbaf9303cc842888d242f6af0cd5"), 5)

In [14]:
print_program(genesis_coin_spend.puzzle_reveal.to_program())
print_program(genesis_coin_spend.solution.to_program())
print_program(genesis_coin_spend.puzzle_reveal.to_program().run(genesis_coin_spend.solution.to_program()))
print_conditions(
    genesis_coin_spend.puzzle_reveal.to_program(),
    genesis_coin_spend.solution.to_program()
)

(a (q 2 (q 2 (i 11 (q 2 (i (= 5 (point_add 11 (pubkey_for_exp (sha256 11 (a 6 (c 2 (c 23 ()))))))) (q 2 23 47) (q  
8)) 1) (q 4 (c 4 (c 5 (c (a 6 (c 2 (c 23 ()))) ()))) (a 23 47))) 1) (c (q 50 2 (i (l 5) (q 11 (q . 2) (a 6 (c 2 (c 
9 ()))) (a 6 (c 2 (c 13 ())))) (q 11 (q . 1) 5)) 1) 1)) (c (q .                                                    
0xaf14d22fba1ab9a63ec96f2f53cc4039fba865ab73753fb6ac9f8d397c864eb60349677ab155c355ed47f9a3e8e7bbaf) 1))            

(() (q (51 0x71f0c6b06309d6582e247b523fd42d8cedf6e3c69f583619a2dbc9dda80827a2 0x174876e800) (51                    
0x52b7e40c813355fbc9393c08bb3c472584f0dd8c56910c50d20e538760899e12 0x0474dec0e160) (52 0x0186a0) (60               
0xcfe081ecf6778c82af2cdb60831508d7edcc4167da046771edbafbf7541f5b40)) ())                                           

((50 0xaf14d22fba1ab9a63ec96f2f53cc4039fba865ab73753fb6ac9f8d397c864eb60349677ab155c355ed47f9a3e8e7bbaf            
0x8699e090da8384d47c4120d9b79cf3812a1dbf655662d35081558a17d40e5c2f) (51                                            
0x71f0c6b06309d6582e247b523fd42d8cedf6e3c69f583619a2dbc9dda80827a2 0x174876e800) (51                               
0x52b7e40c813355fbc9393c08bb3c472584f0dd8c56910c50d20e538760899e12 0x0474dec0e160) (52 0x0186a0) (60               
0xcfe081ecf6778c82af2cdb60831508d7edcc4167da046771edbafbf7541f5b40))                                               

(AGG_SIG_ME 0xaf14d22fba1ab9a63ec96f2f53cc4039fba865ab73753fb6ac9f8d397c864eb60349677ab155c355ed47f9a3e8e7bbaf     
0x8699e090da8384d47c4120d9b79cf3812a1dbf655662d35081558a17d40e5c2f)                                                

(CREATE_COIN 0x71f0c6b06309d6582e247b523fd42d8cedf6e3c69f583619a2dbc9dda80827a2 0x174876e800)                      

(CREATE_COIN 0x52b7e40c813355fbc9393c08bb3c472584f0dd8c56910c50d20e538760899e12 0x0474dec0e160)                    

(RESERVE_FEE 0x0186a0)                                                                                             

(CREATE_COIN_ANNOUNCEMENT 0xcfe081ecf6778c82af2cdb60831508d7edcc4167da046771edbafbf7541f5b40)                      

In [15]:
eve_coin_spend = await get_coin_spend(bytes32.from_hexstr("9faaec9124a3538cb7977ceea984bdb023914ac1e7bfa0329e3fa11ebbfe1e1a"), 5)

In [16]:
print_program(eve_coin_spend.puzzle_reveal.to_program())
print_program(eve_coin_spend.solution.to_program())

print_program(eve_coin_spend.puzzle_reveal.to_program().run(eve_coin_spend.solution.to_program()))
print_conditions(
    eve_coin_spend.puzzle_reveal.to_program(),
    eve_coin_spend.solution.to_program()
)

(a (q 2 (q 2 94 (c 2 (c (c 5 (c (sha256 52 5) (c 11 ()))) (c (a 23 47) (c 95 (c (a 46 (c 2 (c 23 ()))) (c (a 42 (c 
2 (c 639 (c 1407 (c 2943 ()))))) (c -65 (c 383 (c 767 (c 1535 (c 3071 ())))))))))))) (c (q (((61 . 70) 2 51 . 60)  
(c . 1) 1 -53 . 2) ((not 2 (i 5 (q 2 50 (c 2 (c 13 (c (sha256 124 (sha256 52 36) (sha256 124 (sha256 124 (sha256 52
44) 9) (sha256 124 11 (sha256 52 ())))) ())))) (q . 11)) 1) (a (i (all (= (strlen 5) 34) (= (strlen 11) 34) (> 23  
(q . -1))) (q 11 5 11 23) (q 8)) 1) (a (i 11 (q 2 (i (= (a 46 (c 2 (c 19 ()))) 2975) (q 2 86

(() () 0x9faaec9124a3538cb7977ceea984bdb023914ac1e7bfa0329e3fa11ebbfe1e1a                                          
(0x0912e598c7e197a296283436bde9594fdc40fbaf9303cc842888d242f6af0cd5                                                
0x71f0c6b06309d6582e247b523fd42d8cedf6e3c69f583619a2dbc9dda80827a2 0x174876e800)                                   
(0x0912e598c7e197a296283436bde9594fdc40fbaf9303cc842888d242f6af0cd5                                                
0xa621bf2c0c4305e9b3079cf3191b4b01569261a844d44d63e02381eebb651c21 0x174876e800) () ())                            

((70 0x9faaec9124a3538cb7977ceea984bdb023914ac1e7bfa0329e3fa11ebbfe1e1a) (60                                       
0xcbf68bf42766be67fd42d52d263e401d314455b43d83f072555c6d5927508d9090) (61                                          
0x51a5ad5671e227b6d1f3e0e1757cd4ff0874587d5a38f805d9733358bbaee763) (51                                            
0xbe9fc5a1356babff14efd8eb7ffaf152f221c402d71fcca0f0d216cd129c1c70 0x174876e800                                    
(0x1cb70b7a0b0670102d0aea6db180ffe2375cea7e685fcebfd8f54482e0d5049f)))                                             

(ASSERT_MY_COIN_ID 0x9faaec9124a3538cb7977ceea984bdb023914ac1e7bfa0329e3fa11ebbfe1e1a)                             

(CREATE_COIN_ANNOUNCEMENT 0xcbf68bf42766be67fd42d52d263e401d314455b43d83f072555c6d5927508d9090)                    

(ASSERT_COIN_ANNOUNCEMENT 0x51a5ad5671e227b6d1f3e0e1757cd4ff0874587d5a38f805d9733358bbaee763)                      

(CREATE_COIN 0xbe9fc5a1356babff14efd8eb7ffaf152f221c402d71fcca0f0d216cd129c1c70 0x174876e800)                      

In [8]:
%%bash
chia show -s
chia wallet show -f 236020623 -w cat

Network: simulator0    Port: 50090   RPC Port: 16909
Node ID: 9d1c999ccfb4313223a6af62695026a0a3d0ce5c3ca8034283b3ded7651f1007
Genesis Challenge: eb8c4d20b322be8d9fddbf9412016bdffe9a2901d7edb0e364e94266d0e095f7
Current Blockchain Status: Full Node Synced

Peak: Hash: 3dda8c9bcd5953a50f9b9130ce5df5c80ec4ef6012704b45a8c9a17e351da9c1
      Time: Wed Jan 11 2023 09:19:50 CST                  Height:          5

Estimated network space: 48.250 MiB
Current difficulty: 1024
Current VDF sub_slot_iters: 1024

  Height: |   Hash:
        5 | 3dda8c9bcd5953a50f9b9130ce5df5c80ec4ef6012704b45a8c9a17e351da9c1
        4 | 03fb75348b0ed95f0791c821a714edcbd8ad551402e330b296b8cfd5062e62c8
        3 | 3f3dbf30047447cb9a11bd8707f3af5f0f93663a0157c10c9494d239a7efb24e
        2 | d690e9eee15e67078c6a09abebcdbe690622076261d7318bd2f91324d4c2ceb3
        1 | b4abff61105de1aaf1a3b527d776b7cff44d0710a46997e42623c6904f13e4cf
Wallet height: 5
Sync status: Synced
Balances, fingerprint: 236020623

SDBX:
   -Total Ba

## SDBX Coin


In [15]:
%%bash
cdv decode txch1rjmsk7stqecpqtg2afkmrq8lugm4e6n7dp0ua07c74zg9cx4qj0sg0e5l9
cdv clsp cat_puzzle_hash -t a1cdd796a588491663f61dfe9305eb67e8fed0e016961da3958862c68589fd41 1cb70b7a0b0670102d0aea6db180ffe2375cea7e685fcebfd8f54482e0d5049f

1cb70b7a0b0670102d0aea6db180ffe2375cea7e685fcebfd8f54482e0d5049f
be9fc5a1356babff14efd8eb7ffaf152f221c402d71fcca0f0d216cd129c1c70


In [16]:
%%bash
cdv rpc coinrecords --by puzzlehash be9fc5a1356babff14efd8eb7ffaf152f221c402d71fcca0f0d216cd129c1c70 -nd

{
    "01fc8146a15d1c089154902b8703852a7e0a42f151d2bbfa87e276e845d1a7de": {
        "coin": {
            "amount": 100000000000,
            "parent_coin_info": "0x9faaec9124a3538cb7977ceea984bdb023914ac1e7bfa0329e3fa11ebbfe1e1a",
            "puzzle_hash": "0xbe9fc5a1356babff14efd8eb7ffaf152f221c402d71fcca0f0d216cd129c1c70"
        },
        "coinbase": false,
        "confirmed_block_index": 5,
        "spent_block_index": 0,
        "timestamp": 1673450390
    }
}


In [21]:
await get_hints()

coin_id: 01fc8146a15d1c089154902b8703852a7e0a42f151d2bbfa87e276e845d1a7de

hint:    1cb70b7a0b0670102d0aea6db180ffe2375cea7e685fcebfd8f54482e0d5049f

addr:    txch1rjmsk7stqecpqtg2afkmrq8lugm4e6n7dp0ua07c74zg9cx4qj0sg0e5l9

### lineage_proofs_a1cdd796a588491663f61dfe9305eb67e8fed0e016961da3958862c68589fd41

In [20]:
await get_lineage_proof("lineage_proofs_a1cdd796a588491663f61dfe9305eb67e8fed0e016961da3958862c68589fd41")

9faaec9124a3538cb7977ceea984bdb023914ac1e7bfa0329e3fa11ebbfe1e1a

1cb70b7a0b0670102d0aea6db180ffe2375cea7e685fcebfd8f54482e0d5049f

100000000000

0912e598c7e197a296283436bde9594fdc40fbaf9303cc842888d242f6af0cd5

a621bf2c0c4305e9b3079cf3191b4b01569261a844d44d63e02381eebb651c21

100000000000

## Spend SDBX Coin
```sh
❯ chia wallet send -f $single_fp -t txch122m7grypxd2lhjfe8sytk0z8ykz0phvv26gsc5xjpefcwcyfncfqsx8w2w -i 2 -a 2500
Submitting transaction...
Transaction submitted to nodes: [{'peer_id': '9d1c999ccfb4313223a6af62695026a0a3d0ce5c3ca8034283b3ded7651f1007', 'inclusion_status': 'SUCCESS', 'error_msg': None}]
Run 'chia wallet get_transaction -f 236020623 -tx 0x316b13ebc80f595b55a8d71c4aea178e0c43bde61e9317ab33e268f60cd6ee66' to get status
```

In [4]:
await get_hints()

coin_id: 01fc8146a15d1c089154902b8703852a7e0a42f151d2bbfa87e276e845d1a7de

hint:    1cb70b7a0b0670102d0aea6db180ffe2375cea7e685fcebfd8f54482e0d5049f

addr:    txch1rjmsk7stqecpqtg2afkmrq8lugm4e6n7dp0ua07c74zg9cx4qj0sg0e5l9

coin_id: f02b2a5b3b7e9ea0164de344114f4eec28506c7628cb0fa3dbb57a955add9963

hint:    52b7e40c813355fbc9393c08bb3c472584f0dd8c56910c50d20e538760899e12

addr:    txch122m7grypxd2lhjfe8sytk0z8ykz0phvv26gsc5xjpefcwcyfncfqsx8w2w

In [5]:
await get_lineage_proof("lineage_proofs_a1cdd796a588491663f61dfe9305eb67e8fed0e016961da3958862c68589fd41")

parent_name: 9faaec9124a3538cb7977ceea984bdb023914ac1e7bfa0329e3fa11ebbfe1e1a

inner_ph: 1cb70b7a0b0670102d0aea6db180ffe2375cea7e685fcebfd8f54482e0d5049f

amount: 100000000000

parent_name: 0912e598c7e197a296283436bde9594fdc40fbaf9303cc842888d242f6af0cd5

inner_ph: a621bf2c0c4305e9b3079cf3191b4b01569261a844d44d63e02381eebb651c21

...

inner_ph: 4a6560cd415a8113a457c3c98dbdb58e0f0aeb54059af40d3ffde3e928ebd260

amount: 99997500000

parent_name: 01fc8146a15d1c089154902b8703852a7e0a42f151d2bbfa87e276e845d1a7de

inner_ph: 52b7e40c813355fbc9393c08bb3c472584f0dd8c56910c50d20e538760899e12

amount: 2500000

#### New Hint
```yaml
coin_id: f02b2a5b3b7e9ea0164de344114f4eec28506c7628cb0fa3dbb57a955add9963
hint:    52b7e40c813355fbc9393c08bb3c472584f0dd8c56910c50d20e538760899e12
```

### New Lineage Proofs

1. 2_500_000 mojos sent to `txch122m7grypxd2lhjfe8sytk0z8ykz0phvv26gsc5xjpefcwcyfncfqsx8w2w`

2. 99_997_500_000 mojos (changes) sent to `txch1ffjkpn2pt2q38fzhc0ycm0d43c8s4665qkd0grfllh37j28t6fsqtarg0y`

```yaml
parent_name: 01fc8146a15d1c089154902b8703852a7e0a42f151d2bbfa87e276e845d1a7de
inner_ph: 4a6560cd415a8113a457c3c98dbdb58e0f0aeb54059af40d3ffde3e928ebd260
amount: 99997500000
parent_name: 01fc8146a15d1c089154902b8703852a7e0a42f151d2bbfa87e276e845d1a7de
inner_ph: 52b7e40c813355fbc9393c08bb3c472584f0dd8c56910c50d20e538760899e12
amount: 2500000
```
```sh
❯ cdv encode 4a6560cd415a8113a457c3c98dbdb58e0f0aeb54059af40d3ffde3e928ebd260 --prefix txch
txch1ffjkpn2pt2q38fzhc0ycm0d43c8s4665qkd0grfllh37j28t6fsqtarg0y

❯ cdv encode 52b7e40c813355fbc9393c08bb3c472584f0dd8c56910c50d20e538760899e12 --prefix txch
txch122m7grypxd2lhjfe8sytk0z8ykz0phvv26gsc5xjpefcwcyfncfqsx8w2w
```

In [17]:
cat_coin_spend = await get_coin_spend(bytes32.from_hexstr("01fc8146a15d1c089154902b8703852a7e0a42f151d2bbfa87e276e845d1a7de"), 7)
print(cat_coin_spend.coin)
print_program(cat_coin_spend.puzzle_reveal.to_program())
print_program(cat_coin_spend.solution.to_program())
print_program(cat_coin_spend.puzzle_reveal.to_program().run(cat_coin_spend.solution.to_program()))
print_conditions(
    cat_coin_spend.puzzle_reveal.to_program(),
    cat_coin_spend.solution.to_program()
)

Coin { parent_coin_info: 9faaec9124a3538cb7977ceea984bdb023914ac1e7bfa0329e3fa11ebbfe1e1a, puzzle_hash: 
be9fc5a1356babff14efd8eb7ffaf152f221c402d71fcca0f0d216cd129c1c70, amount: 100000000000 }

(a (q 2 (q 2 94 (c 2 (c (c 5 (c (sha256 52 5) (c 11 ()))) (c (a 23 47) (c 95 (c (a 46 (c 2 (c 23 ()))) (c (a 42 (c 
2 (c 639 (c 1407 (c 2943 ()))))) (c -65 (c 383 (c 767 (c 1535 (c 3071 ())))))))))))) (c (q (((61 . 70) 2 51 . 60)  
(c . 1) 1 -53 . 2) ((not 2 (i 5 (q 2 50 (c 2 (c 13 (c (sha256 124 (sha256 52 36) (sha256 124 (sha256 124 (sha256 52
44) 9) (sha256 124 11 (sha256 52 ())))) ())))) (q . 11)) 1) (a (i (all (= (strlen 5) 34) (= (strlen 11) 34) (> 23  
(q . -1))) (q 11 5 11 23) (q 8)) 1) (a (i 11 (q 2 (i (= (a 46 (c 2 (c 19 ()))) 2975) (q 2 86

((() (q (51 0x52b7e40c813355fbc9393c08bb3c472584f0dd8c56910c50d20e538760899e12 0x2625a0                            
(0x52b7e40c813355fbc9393c08bb3c472584f0dd8c56910c50d20e538760899e12)) (51                                          
0x4a6560cd415a8113a457c3c98dbdb58e0f0aeb54059af40d3ffde3e928ebd260 0x174850c260) (60                               
0xe6b727a87e284770eee6737b5ecf6d37d0e3ca253077f8cb156432df17604075)) ())                                           
(0x0912e598c7e197a296283436bde9594fdc40fbaf9303cc842888d242f6af0cd5                                                
0xa621bf2c0c4305e9b3079cf3191b4b01569261a844d44d63e02381eebb651c21 0x174876e800)                                   
0x01fc8146a15d1c089154902b8703852a7e0a42f151d2bbfa87e276e845d1a7de                                                 
(0x9faaec9124a3538cb7977ceea984bdb023914ac1e7bfa0329e3fa11ebbfe1e1a                                                
0xbe9fc5a1356babff14efd8eb7ffaf152f221c402d71fcca0f0d216cd129c1c70 0x174876e800)                                   
(0x9faaec9124a3538cb7977ceea984bdb023914ac1e7bfa0329e3fa11ebbfe1e1a                                                
0x1cb70b7a0b0670102d0aea6db180ffe2375cea7e685fcebfd8f54482e0d5049f 0x174876e800) () ())                            

((70 0x01fc8146a15d1c089154902b8703852a7e0a42f151d2bbfa87e276e845d1a7de) (60                                       
0xcb52e9bf88fc91a67d117775b6f8ab297b8ec8c345f756260d370cbd8befd81f3f) (61                                          
0xa2dc86a0d3931cf4f422143b4177b637c3f0cc6f0a8a968856ba6c0adf89e938) (50                                            
0xaf14d22fba1ab9a63ec96f2f53cc4039fba865ab73753fb6ac9f8d397c864eb60349677ab155c355ed47f9a3e8e7bbaf                 
0x4ce14931b9b91397177e3e28d66cbaf123aeb73868fb1bc1aa0260713c92f335) (51                                            
0xae90aba05c59b9b0f9e07755b9358de918c51cedea67ccdf7d92352e69a5d7e6 0x2625a0                                        
(0x52b7e40c813355fbc9393c08bb3c472584f0dd8c56910c50d20e538760899e12)) (51                                          
0x8f19f2d55e51179ecd27b91d0ccc7de6fd36e8fb2cd5f9d9f22c2095770d9d6c 0x174850c260) (60                               
0xe6b727a87e284770eee6737b5ecf6d37d0e3ca253077f8cb156432df17604075))                                               

(ASSERT_MY_COIN_ID 0x01fc8146a15d1c089154902b8703852a7e0a42f151d2bbfa87e276e845d1a7de)                             

...

(ASSERT_COIN_ANNOUNCEMENT 0xa2dc86a0d3931cf4f422143b4177b637c3f0cc6f0a8a968856ba6c0adf89e938)                      

(AGG_SIG_ME 0xaf14d22fba1ab9a63ec96f2f53cc4039fba865ab73753fb6ac9f8d397c864eb60349677ab155c355ed47f9a3e8e7bbaf     
0x4ce14931b9b91397177e3e28d66cbaf123aeb73868fb1bc1aa0260713c92f335)                                                

(CREATE_COIN 0xae90aba05c59b9b0f9e07755b9358de918c51cedea67ccdf7d92352e69a5d7e6 0x2625a0)                          

(CREATE_COIN 0x8f19f2d55e51179ecd27b91d0ccc7de6fd36e8fb2cd5f9d9f22c2095770d9d6c 0x174850c260)                      

(CREATE_COIN_ANNOUNCEMENT 0xe6b727a87e284770eee6737b5ecf6d37d0e3ca253077f8cb156432df17604075)                      

In [18]:
# (create_announcement_condition prev_coin_id prev_subtotal)

std_hash(
    cat_coin_spend.coin.name() + 
     bytes.fromhex("cb52e9bf88fc91a67d117775b6f8ab297b8ec8c345f756260d370cbd8befd81f3f")
).hex()

'a2dc86a0d3931cf4f422143b4177b637c3f0cc6f0a8a968856ba6c0adf89e938'

In [19]:
std_hash(
    cat_coin_spend.coin.name() + 
     bytes.fromhex("e6b727a87e284770eee6737b5ecf6d37d0e3ca253077f8cb156432df17604075")
).hex()

'be3879fe0c8330055bb167f5862ed6a988f53042d4b3f83238aefe904fa1800c'

In [20]:
# block 7
print(await get_coins_in_block(bytes.fromhex("b6b4e6aa48972e4dc5d4eaee6e028f2906833cf9f4be122f67f3857bb3e19e26")))

(
    [
        CoinRecord(
            coin=Coin { parent_coin_info: 01fc8146a15d1c089154902b8703852a7e0a42f151d2bbfa87e276e845d1a7de, 
puzzle_hash: 8f19f2d55e51179ecd27b91d0ccc7de6fd36e8fb2cd5f9d9f22c2095770d9d6c, amount: 99997500000 },
            confirmed_block_index=7,
            spent_block_index=0,
            coinbase=False,
            timestamp=1673453105
        ),
        CoinRecord(
            coin=Coin { parent_coin_info: 01fc8146a15d1c089154902b8703852a7e0a42f151d2bbfa87e276e845d1a7de, 
puzzle_hash: ae90aba05c59b9b0f9e07755b9358de918c51cedea67ccdf7d92352e69a5d7e6, amount: 2500000 },
            confirmed_block_index=7,
            spent_block_index=0,
            coinbase=False,
            timestamp=1673453105
        ),
        CoinRecord(
            coin=Coin { parent_coin_info: eb8c4d20b322be8d9fddbf9412016bdf00000000000000000000000000000006, 
puzzle_hash: 3e68916b07065876f25b0f7712fd68b975a779b5b56735b266db8aa793841535, amount: 1750000000000 },
            confirmed_block_index=7,
            spent_block_index=0,
            coinbase=True,
            timestamp=1673453105
        ),
        CoinRecord(
            coin=Coin { parent_coin_info: fe9a2901d7edb0e364e94266d0e095f700000000000000000000000000000006, 
puzzle_hash: 3e68916b07065876f25b0f7712fd68b975a779b5b56735b266db8aa793841535, amount: 250000000000 },
            confirmed_block_index=7,
            spent_block_index=0,
            coinbase=True,
            timestamp=1673453105
        )
    ],
    [
        CoinRecord(
            coin=Coin { parent_coin_info: 9faaec9124a3538cb7977ceea984bdb023914ac1e7bfa0329e3fa11ebbfe1e1a, 
puzzle_hash: be9fc5a1356babff14efd8eb7ffaf152f221c402d71fcca0f0d216cd129c1c70, amount: 100000000000 },
            confirmed_block_index=5,
            spent_block_index=7,
            coinbase=False,
            timestamp=1673450390
        )
    ]
)

In [21]:
await get_hints()

coin_id: 01fc8146a15d1c089154902b8703852a7e0a42f151d2bbfa87e276e845d1a7de

hint:    1cb70b7a0b0670102d0aea6db180ffe2375cea7e685fcebfd8f54482e0d5049f

addr:    txch1rjmsk7stqecpqtg2afkmrq8lugm4e6n7dp0ua07c74zg9cx4qj0sg0e5l9

coin_id: f02b2a5b3b7e9ea0164de344114f4eec28506c7628cb0fa3dbb57a955add9963

hint:    52b7e40c813355fbc9393c08bb3c472584f0dd8c56910c50d20e538760899e12

addr:    txch122m7grypxd2lhjfe8sytk0z8ykz0phvv26gsc5xjpefcwcyfncfqsx8w2w

```sh
❯ cdv rpc coinrecords --by id f02b2a5b3b7e9ea0164de344114f4eec28506c7628cb0fa3dbb57a955add9963
[
    {
        "coin": {
            "amount": 2500000,
            "parent_coin_info": "0x01fc8146a15d1c089154902b8703852a7e0a42f151d2bbfa87e276e845d1a7de",
            "puzzle_hash": "0xae90aba05c59b9b0f9e07755b9358de918c51cedea67ccdf7d92352e69a5d7e6"
        },
        "coinbase": false,
        "confirmed_block_index": 7,
        "spent_block_index": 0,
        "timestamp": 1673453105
    }
]

❯ cdv rpc coinrecords --by puzzlehash 8f19f2d55e51179ecd27b91d0ccc7de6fd36e8fb2cd5f9d9f22c2095770d9d6c -nd
{
    "f4a98fa94248aa20c0ce30fd5b5a0a7766e1ed8f67e0dccf36cc2d136b906a1c": {
        "coin": {
            "amount": 99997500000,
            "parent_coin_info": "0x01fc8146a15d1c089154902b8703852a7e0a42f151d2bbfa87e276e845d1a7de",
            "puzzle_hash": "0x8f19f2d55e51179ecd27b91d0ccc7de6fd36e8fb2cd5f9d9f22c2095770d9d6c"
        },
        "coinbase": false,
        "confirmed_block_index": 7,
        "spent_block_index": 0,
        "timestamp": 1673453105
    }
}
```

In [22]:
p, c = eve_coin_spend.puzzle_reveal.to_program().uncurry()
print_program(p)
print_program(c)

(a (q 2 94 (c 2 (c (c 5 (c (sha256 52 5) (c 11 ()))) (c (a 23 47) (c 95 (c (a 46 (c 2 (c 23 ()))) (c (a 42 (c 2 (c 
639 (c 1407 (c 2943 ()))))) (c -65 (c 383 (c 767 (c 1535 (c 3071 ())))))))))))) (c (q (((61 . 70) 2 51 . 60) (c .  
1) 1 -53 . 2) ((not 2 (i 5 (q 2 50 (c 2 (c 13 (c (sha256 124 (sha256 52 36) (sha256 124 (sha256 124 (sha256 52 44) 
9) (sha256 124 11 (sha256 52 ())))) ())))) (q . 11)) 1) (a (i (all (= (strlen 5) 34) (= (strlen 11) 34) (> 23 (q . 
-1))) (q 11 5 11 23) (q 8)) 1) (a (i 11 (q 2 (i (= (a 46 (c 2 (c 19 ()))) 2975) (q 2 86 (c [38;2;174;129;255

(0x37bef360ee858133b69d595a906dc45d01af50379dad515eb9518abb7c1d2a7a                                                
0xa1cdd796a588491663f61dfe9305eb67e8fed0e016961da3958862c68589fd41 (q (51 () -113 (a (q 2 (i 47 (q 8) (q 2 (i (= 45
2) () (q 8)) 1)) 1) (c (q . 0x0912e598c7e197a296283436bde9594fdc40fbaf9303cc842888d242f6af0cd5) 1)) ()) (51        
0x1cb70b7a0b0670102d0aea6db180ffe2375cea7e685fcebfd8f54482e0d5049f 0x174876e800                                    
(0x1cb70b7a0b0670102d0aea6db180ffe2375cea7e685fcebfd8f54482e0d5049f))))                                            

In [52]:
p, c = cat_coin_spend.puzzle_reveal.to_program().uncurry()
print_program(p)
print_program(c)

print('Curried')
# cat_v2.clvm
print(f'MOD_HASH: {p.get_tree_hash().hex()}')
# curried genesis_by_coin_id
print(f'TAIL_PROGRAM_HASH: {load_hex("./reference_tails/genesis_by_coin_id.clsp.hex").curry(genesis_coin_id).get_tree_hash()}')
# standard txn with synthetic key
print(f'INNER_PUZZLE:')
print_program(
    puzzle_for_pk(
        G1Element.from_bytes(
            bytes.fromhex("add10d87915c99a2c17df9afb00b2ee2fd636be2b65b97bb0576748586199a7cde0b5244cc72211a65627414af1c3ac4"))))

print('Solution:')
solution = cat_coin_spend.solution.to_program()
print_program(solution)

print('inner_puzzle_solution:')
print_program(solution.first())

print('lineage_proof:')
print_program(solution.at("rf"))

print(f'prev_coin_id: {disassemble(solution.at("rrf"))}')

print('this_coin_info:')
print_program(solution.at("rrrf"))

print('next_coin_proof:')
print_program(solution.at("rrrrf"))

print(f'prev_subtotal: {disassemble(solution.at("rrrrrf"))}')
print(f'extra_delta: {disassemble(solution.at("rrrrrrf"))}')

cat_v2_sig = """
(
      MOD_HASH                 ;; curried into puzzle
      TAIL_PROGRAM_HASH        ;; curried into puzzle
      INNER_PUZZLE             ;; curried into puzzle
      inner_puzzle_solution    ;; if invalid, INNER_PUZZLE will fail
      lineage_proof            ;; This is the parent's coin info, used to check if the parent was a CAT. Optional if using tail_program.
      prev_coin_id             ;; used in this coin's announcement, prev_coin ASSERT_COIN_ANNOUNCEMENT will fail if wrong
      this_coin_info           ;; verified with ASSERT_MY_COIN_ID
      next_coin_proof          ;; used to generate ASSERT_COIN_ANNOUNCEMENT
      prev_subtotal            ;; included in announcement, prev_coin ASSERT_COIN_ANNOUNCEMENT will fail if wrong
      extra_delta              ;; this is the "legal discrepancy" between your real delta and what you're announcing your delta is
)
;; GLOSSARY:
;;  MOD_HASH: this code's sha256 tree hash
;;  TAIL_PROGRAM_HASH: the program that determines if a coin can mint new cats, burn cats, and check if its lineage is valid if its parent is not a CAT
;;  INNER_PUZZLE: an independent puzzle protecting the coins. Solutions to this puzzle are expected to generate `AGG_SIG` conditions and possibly `CREATE_COIN` conditions.
;; ---- items above are curried into the puzzle hash ----
;;  inner_puzzle_solution: the solution to the inner puzzle
;;  prev_coin_id: the id for the previous coin
;;  tail_program_reveal: reveal of TAIL_PROGRAM_HASH required to run the program if desired
;;  tail_solution: optional solution passed into tail_program
;;  lineage_proof: optional proof that our coin's parent is a CAT
;;  this_coin_info: (parent_id puzzle_hash amount)
;;  next_coin_proof: (parent_id inner_puzzle_hash amount)
;;  prev_subtotal: the subtotal between prev-coin and this-coin
;;  extra_delta: an amount that is added to our delta and checked by the TAIL program
;;

"""

(a (q 2 94 (c 2 (c (c 5 (c (sha256 52 5) (c 11 ()))) (c (a 23 47) (c 95 (c (a 46 (c 2 (c 23 ()))) (c (a 42 (c 2 (c 
639 (c 1407 (c 2943 ()))))) (c -65 (c 383 (c 767 (c 1535 (c 3071 ())))))))))))) (c (q (((61 . 70) 2 51 . 60) (c .  
1) 1 -53 . 2) ((not 2 (i 5 (q 2 50 (c 2 (c 13 (c (sha256 124 (sha256 52 36) (sha256 124 (sha256 124 (sha256 52 44) 
9) (sha256 124 11 (sha256 52 ())))) ())))) (q . 11)) 1) (a (i (all (= (strlen 5) 34) (= (strlen 11) 34) (> 23 (q . 
-1))) (q 11 5 11 23) (q 8)) 1) (a (i 11 (q 2 (i (= (a 46 (c 2 (c 19 ()))) 2975) (q 2 86 (c [38;2;174;129;255

(0x37bef360ee858133b69d595a906dc45d01af50379dad515eb9518abb7c1d2a7a                                                
0xa1cdd796a588491663f61dfe9305eb67e8fed0e016961da3958862c68589fd41 (a (q 2 (q 2 (i 11 (q 2 (i (= 5 (point_add 11   
(pubkey_for_exp (sha256 11 (a 6 (c 2 (c 23 ()))))))) (q 2 23 47) (q 8)) 1) (q 4 (c 4 (c 5 (c (a 6 (c 2 (c 23 ()))) 
()))) (a 23 47))) 1) (c (q 50 2 (i (l 5) (q 11 (q . 2) (a 6 (c 2 (c 9 ()))) (a 6 (c 2 (c 13 ())))) (q 11 (q . 1)   
5)) 1) 1)) (c (q .                                                                                                 
0xaf14d22fba1ab9a63ec96f2f53cc4039fba865ab73753fb6ac9f8d397c864eb60349677ab155c355ed47f9a3e8e7bbaf) 1)))           

Curried

MOD_HASH: 37bef360ee858133b69d595a906dc45d01af50379dad515eb9518abb7c1d2a7a

TAIL_PROGRAM_HASH: a1cdd796a588491663f61dfe9305eb67e8fed0e016961da3958862c68589fd41

...

(0x9faaec9124a3538cb7977ceea984bdb023914ac1e7bfa0329e3fa11ebbfe1e1a                                                
0xbe9fc5a1356babff14efd8eb7ffaf152f221c402d71fcca0f0d216cd129c1c70 0x174876e800)                                   

next_coin_proof:

(0x9faaec9124a3538cb7977ceea984bdb023914ac1e7bfa0329e3fa11ebbfe1e1a                                                
0x1cb70b7a0b0670102d0aea6db180ffe2375cea7e685fcebfd8f54482e0d5049f 0x174876e800)                                   

prev_subtotal: ()

extra_delta: ()

MOD_HASH: this code's sha256 tree hash
`37bef360ee858133b69d595a906dc45d01af50379dad515eb9518abb7c1d2a7a`

TAIL_PROGRAM_HASH: the program that determines if a coin can mint new cats, burn cats, and check if its lineage is valid if its parent is not a CAT
`a1cdd796a588491663f61dfe9305eb67e8fed0e016961da3958862c68589fd41`

INNER_PUZZLE: an independent puzzle protecting the coins. Solutions to this puzzle are expected to generate `AGG_SIG` conditions and possibly `CREATE_COIN` conditions.
```sh
❯ chia keys derive -f $single_fp child-key -n 1 -t wallet
Wallet public key 0: add10d87915c99a2c17df9afb00b2ee2fd636be2b65b97bb0576748586199a7cde0b5244cc72211a65627414af1c3ac4

~
❯ cdv inspect keys -pk add10d87915c99a2c17df9afb00b2ee2fd636be2b65b97bb0576748586199a7cde0b5244cc72211a65627414af1c3ac4 -sy
Public Key: af14d22fba1ab9a63ec96f2f53cc4039fba865ab73753fb6ac9f8d397c864eb60349677ab155c355ed47f9a3e8e7bbaf
Fingerprint: 1786593521
HD Path: m
```